<a href="https://colab.research.google.com/github/agrr12/TCC/blob/master/TCC3_post_distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.functions import *
from google.colab import drive
from pyspark.sql.types import *
import numpy as np
from scipy import sparse
from datetime import *
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr
import warnings
import seaborn as sns
from PIL import Image
from collections import Counter
from matplotlib.ticker import MaxNLocator
from scipy.spatial.distance import cosine


ERROR: Operation cancelled by user
^C


In [ ]:
spark = SparkSession.builder.master("local[*]").appName('TCC').getOrCreate()
sc = spark.sparkContext
file_name = 'date_array.csv'
drive_path= 'My Drive/TCC'
drive.mount('/content/drive')
df = pd.read_csv(f'/content/drive/{drive_path}/{file_name}')
df.head(5)

Mounted at /content/drive


,authorChannelId,collect_list(CAST(publishedAt AS STRING))
0,UC-K0pQo4HP0IjwDc65X4Zng,"['2022-10-31 14:13:39', '2022-11-02 15:58:05',..."
1,UC042BoatnB2tts9IwldTJpw,"['2022-11-05 23:21:35', '2022-09-27 20:37:52',..."
2,UC0BXL2bOfLSONFDQSAFzyAQ,"['2022-10-29 21:31:35', '2022-10-31 14:16:13',..."
3,UC0DTrr8CZEtrjId6h4erx1A,"['2022-10-30 14:42:32', '2022-11-03 23:31:21',..."
4,UC0MuRWrWdjZTLZCgLA6d_xQ,"['2022-10-22 02:33:47', '2022-09-12 21:51:47',..."


In [ ]:
def create_sparse_matrix(dates, start_date_str: str, end_date_str: str, unit: str = 'seconds') -> sparse.dok_matrix:
    """
    Create a sparse matrix based on the difference between two date strings.

    Parameters:
    - start_date_str (str): Start date in the format '%Y-%m-%d %H:%M:%S'.
    - end_date_str (str): End date in the format '%Y-%m-%d %H:%M:%S'.
    - unit (str): Desired unit for the matrix columns ('seconds', 'minutes', or 'hours').

    Returns:
    - sparse.dok_matrix: A matrix with shape (days, units_per_day).
    """
    # Define the date format
    fmt = '%Y-%m-%d %H:%M:%S'

    # Parse the start and end dates
    d1 = datetime.strptime(start_date_str, fmt)
    d2 = datetime.strptime(end_date_str, fmt)

    # Calculate the difference in days
    delta = d2 - d1
    days = delta.days + 1  # +1 to include the last day

    # Define units per day
    if unit == 'seconds':
        units_per_day = 24 * 60 * 60
    elif unit == 'minutes':
        units_per_day = 24 * 60
    elif unit == 'hours':
        units_per_day = 24

    # Create a sparse matrix with zeros
    sparse_matrix = sparse.dok_matrix((days, units_per_day), dtype=np.int8)
    for d in dates:
      day = (datetime.strptime(d, '%Y-%m-%d %H:%M:%S') - datetime.strptime(start_date_str, '%Y-%m-%d %H:%M:%S')).days
      datetime_d = datetime.strptime(d, '%Y-%m-%d %H:%M:%S')
      if unit == 'seconds':
          index = datetime_d.hour * 3600 + datetime_d.minute * 60 + datetime_d.second
      elif unit == 'minutes':
          index = datetime_d.hour * 60 + datetime_d.minute
      elif unit == 'hours':
          index = datetime_d.hour
      sparse_matrix[day-1, index-1] += 1
    return sparse_matrix

In [ ]:
data = []
for x,y in df.iterrows():
  print(x)
  l = eval(y['collect_list(CAST(publishedAt AS STRING))'])
  a = y['authorChannelId']
  m = create_sparse_matrix(l, '2022-09-01 00:00:01', '2022-11-30 23:59:59','hours').toarray()
  # Compute the distribution for days by summing across hours
  days_distribution = np.sum(m, axis=1)
  # Compute the distribution for hours by summing across days
  hours_distribution = np.sum(m, axis=0)
  final = days_distribution.tolist() + hours_distribution.tolist()
  print(x, len(days_distribution), len(hours_distribution))
  data.append((a, final))


df_pandas = pd.DataFrame(data, columns=['authorChannelId', 'post_distribution'])
df_pandas.to_csv(f"/content/drive/My Drive/TCC/post_distribution_merged.csv", index=False, sep=";")
df_pandas.head(10)

0
0 91 24
1
1 91 24
2
2 91 24
3
3 91 24
4
4 91 24
5
5 91 24
6
6 91 24
7
7 91 24
8
8 91 24
9
9 91 24
10
10 91 24
11
11 91 24
12
12 91 24
13
13 91 24
14
14 91 24
15
15 91 24
16
16 91 24
17
17 91 24
18
18 91 24
19
19 91 24
20
20 91 24
21
21 91 24
22
22 91 24
23
23 91 24
24
24 91 24
25
25 91 24
26
26 91 24
27
27 91 24
28
28 91 24
29
29 91 24
30
30 91 24
31
31 91 24
32
32 91 24
33
33 91 24
34
34 91 24
35
35 91 24
36
36 91 24
37
37 91 24
38
38 91 24
39
39 91 24
40
40 91 24
41
41 91 24
42
42 91 24
43
43 91 24
44
44 91 24
45
45 91 24
46
46 91 24
47
47 91 24
48
48 91 24
49
49 91 24
50
50 91 24
51
51 91 24
52
52 91 24
53
53 91 24
54
54 91 24
55
55 91 24
56
56 91 24
57
57 91 24
58
58 91 24
59
59 91 24
60
60 91 24
61
61 91 24
62
62 91 24
63
63 91 24
64
64 91 24
65
65 91 24
66
66 91 24
67
67 91 24
68
68 91 24
69
69 91 24
70
70 91 24
71
71 91 24
72
72 91 24
73
73 91 24
74
74 91 24
75
75 91 24
76
76 91 24
77
77 91 24
78
78 91 24
79
79 91 24
80
80 91 24
81
81 91 24
82
82 91 24
83
83 91 24
84
84 91 24


,authorChannelId,post_distribution
0,UC-K0pQo4HP0IjwDc65X4Zng,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,UC042BoatnB2tts9IwldTJpw,"[0, 18, 13, 9, 18, 15, 0, 9, 0, 0, 0, 0, 0, 9,..."
2,UC0BXL2bOfLSONFDQSAFzyAQ,"[17, 2, 4, 7, 18, 27, 1, 5, 4, 12, 6, 10, 12, ..."
3,UC0DTrr8CZEtrjId6h4erx1A,"[2, 11, 11, 2, 9, 4, 1, 13, 7, 6, 4, 5, 5, 3, ..."
4,UC0MuRWrWdjZTLZCgLA6d_xQ,"[17, 5, 2, 0, 4, 1, 14, 25, 11, 6, 14, 9, 1, 2..."
5,UC1JDIWa7ydNkrTqJU98Qz9g,"[9, 7, 1, 8, 11, 25, 6, 19, 17, 3, 5, 9, 12, 9..."
6,UC1PPF-cSCAgdmApk1t7Wung,"[6, 2, 2, 0, 0, 3, 0, 2, 2, 2, 3, 1, 2, 1, 2, ..."
7,UC1hqzN6WWy_qNXDZAVk5EEw,"[0, 0, 1, 0, 4, 8, 31, 6, 3, 15, 5, 11, 0, 9, ..."
8,UC1iw3TmZsFamKz7YpPh1Wig,"[3, 4, 4, 0, 2, 2, 4, 7, 5, 3, 6, 6, 5, 0, 1, ..."
9,UC1rrVrDilPXH43jt2IOUN5Q,"[5, 24, 19, 2, 30, 17, 4, 4, 8, 8, 3, 12, 16, ..."


In [ ]:
data = []

for x1, y1 in df_pandas.iterrows():
    a1 = y1['authorChannelId']
    post_distribution_1 = np.array(y1['post_distribution'])

    for x2, y2 in df_pandas[x1+1::].iterrows():
        print(x1, x2)

        a2 = y2['authorChannelId']
        post_distribution_2 = np.array(y2['post_distribution'])

        euclidean_dist = np.linalg.norm(post_distribution_1 - post_distribution_2)
        manhattan_dist = np.sum(np.abs(post_distribution_1 - post_distribution_2))
        cosine_sim = cosine(post_distribution_1, post_distribution_2)
        pearson_corr = np.corrcoef(post_distribution_1, post_distribution_2)[0, 1]

        data.append((a1, a2, euclidean_dist, manhattan_dist, cosine_sim, pearson_corr))

df_pandas2 = pd.DataFrame(data, columns=['author1', 'author2', 'euclidean_distance', 'manhattan_distance', 'cosine_similarity', 'pearson_correlation'])
df_pandas2.to_csv(f"/content/drive/My Drive/TCC/metrics_post_distribution_merged.csv", index=False, sep=",")
df_pandas2.head(10)


A saída de streaming foi truncada nas últimas 5000 linhas.
581 632
581 633
581 634
581 635
581 636
581 637
581 638
581 639
581 640
581 641
581 642
581 643
581 644
581 645
581 646
581 647
581 648
581 649
581 650
581 651
581 652
581 653
581 654
581 655
581 656
581 657
581 658
581 659
581 660
581 661
581 662
581 663
581 664
581 665
581 666
581 667
581 668
581 669
581 670
581 671
581 672
581 673
581 674
581 675
581 676
581 677
581 678
581 679
581 680
581 681
582 583
582 584
582 585
582 586
582 587
582 588
582 589
582 590
582 591
582 592
582 593
582 594
582 595
582 596
582 597
582 598
582 599
582 600
582 601
582 602
582 603
582 604
582 605
582 606
582 607
582 608
582 609
582 610
582 611
582 612
582 613
582 614
582 615
582 616
582 617
582 618
582 619
582 620
582 621
582 622
582 623
582 624
582 625
582 626
582 627
582 628
582 629
582 630
582 631
582 632
582 633
582 634
582 635
582 636
582 637
582 638
582 639
582 640
582 641
582 642
582 643
582 644
582 645
582 646
582 647
582 648
582 649
582 6

,author1,author2,euclidean_distance,manhattan_distance,cosine_similarity,pearson_correlation
0,UC-K0pQo4HP0IjwDc65X4Zng,UC042BoatnB2tts9IwldTJpw,283.372546,1994,0.425411,0.346471
1,UC-K0pQo4HP0IjwDc65X4Zng,UC0BXL2bOfLSONFDQSAFzyAQ,300.106648,2322,0.351252,0.434921
2,UC-K0pQo4HP0IjwDc65X4Zng,UC0DTrr8CZEtrjId6h4erx1A,283.929569,1922,0.430459,0.376494
3,UC-K0pQo4HP0IjwDc65X4Zng,UC0MuRWrWdjZTLZCgLA6d_xQ,249.695815,1702,0.260759,0.588727
4,UC-K0pQo4HP0IjwDc65X4Zng,UC1JDIWa7ydNkrTqJU98Qz9g,325.262971,2304,0.630816,0.026330
5,UC-K0pQo4HP0IjwDc65X4Zng,UC1PPF-cSCAgdmApk1t7Wung,258.600077,1718,0.268929,0.572312
6,UC-K0pQo4HP0IjwDc65X4Zng,UC1hqzN6WWy_qNXDZAVk5EEw,254.754784,1748,0.296181,0.553134
7,UC-K0pQo4HP0IjwDc65X4Zng,UC1iw3TmZsFamKz7YpPh1Wig,280.773930,1924,0.383386,0.379497
8,UC-K0pQo4HP0IjwDc65X4Zng,UC1rrVrDilPXH43jt2IOUN5Q,272.976189,1964,0.369510,0.416158
9,UC-K0pQo4HP0IjwDc65X4Zng,UC1ru5NEMMr6n1HyL_o_a7oQ,348.413547,2294,0.572986,0.276139
